# データ可視化のプロセス：ノーベル賞受賞者のデータ

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_2/nobel-laureates.ipynb)


Wikipediaからスクレイピングしてきたノーベル賞受賞者のデータを使って、データ可視化の6ステップをたどります。

In [35]:
# ライブラリをロードする
import pandas as pd
import numpy as np

## 問いの設定

歴代ノーベル賞受賞者の属性に共通するパターンはあるか？

## データの取得

スクレイピングはこの講義のスコープを超えるので、手元にすでにスクレイピング済みのデータがあると仮定します。

PythonによるWebスクレイピングに有用なライブラリとして、下記があります。

* [requests](https://requests-docs-ja.readthedocs.io/en/latest/): HTTP通信用のライブラリ
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): XMLパース用のライブラリ
* [Scrapy](https://scrapy.org/): スクレイピング、クローリング用のフレームワーク

In [3]:
# JSON形式のファイルを読み込む

file_url = "https://raw.githubusercontent.com/shinchu/dataviz-notebooks/main/data/week_2/nobel_winners_dirty.json"
df = pd.read_json(file_url)

### データの確認

In [10]:
# まずはデータの形をざっと確認

df.head()

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year
0,,Physiology or Medicine,Argentina,8 October 1927,24 March 2002,male,http://en.wikipedia.org/wiki/C%C3%A9sar_Milstein,César Milstein,"Bahía Blanca , Argentina","Cambridge , England","César Milstein , Physiology or Medicine, 1984",1984
1,Bosnia and Herzegovina,Literature,,9 October 1892,13 March 1975,male,http://en.wikipedia.org/wiki/Ivo_Andric,Ivo Andric *,"Dolac (village near Travnik), Austria-Hungary ...","Belgrade, SR Serbia, SFR Yugoslavia (present-d...","Ivo Andric *, born in then Austria–Hungary ,...",1961
2,Bosnia and Herzegovina,Chemistry,,"July 23, 1906",1998-01-07,male,http://en.wikipedia.org/wiki/Vladimir_Prelog,Vladimir Prelog *,"Sarajevo , Bosnia and Herzegovina , then part...","Zürich , Switzerland","Vladimir Prelog *, born in then Austria–Hung...",1975
3,,Peace,Belgium,None,None,None,http://en.wikipedia.org/wiki/Institut_de_Droit...,Institut de Droit International,None,None,"Institut de Droit International , Peace, 1904",1904
4,,Peace,Belgium,26 July 1829,6 October 1912,male,http://en.wikipedia.org/wiki/Auguste_Marie_Fra...,Auguste Beernaert,"Ostend , Netherlands (now Belgium )","Lucerne , Switzerland","Auguste Beernaert , Peace, 1909",1909


In [11]:
# 列を見る

df.columns

Index(['born_in', 'category', 'country', 'date_of_birth', 'date_of_death',
       'gender', 'link', 'name', 'place_of_birth', 'place_of_death', 'text',
       'year'],
      dtype='object')

In [12]:
# 行を見る

df.index

RangeIndex(start=0, stop=1052, step=1)

In [21]:
# 概要を見る

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   born_in         1052 non-null   object
 1   category        1052 non-null   object
 2   country         1052 non-null   object
 3   date_of_birth   1044 non-null   object
 4   date_of_death   1044 non-null   object
 5   gender          1040 non-null   object
 6   link            1052 non-null   object
 7   name            1052 non-null   object
 8   place_of_birth  1044 non-null   object
 9   place_of_death  1044 non-null   object
 10  text            1052 non-null   object
 11  year            1052 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 98.8+ KB


ここから、一部のフィールド（性別など）が欠損していることが分かります。

また、読み込んだばかりの状態では、`year`のみが数値（`int64`）になっており、他の変数はオブジェクトとして扱われていることが分かります。オブジェクトはpandasのデフォルトのデータ型で、任意の数値、文字列、日時などを表すことができます。

In [25]:
# 要約統計量を見る

df.describe()

,year
count,1052.000000
mean,1968.729087
std,33.155829
min,1809.000000
25%,1947.000000
50%,1975.000000
75%,1996.000000
max,2014.000000


デフォルトの状態では、数値列だけが表示されます。

最小値は1809ですが、ノーベル賞の授与は1901年から始まったので明らかに正しくない数値が含まれていることが分かります。最大値は2014で、2014年までの受賞者が含まれているようです。

`describe`にパラメータを与えることで、オブジェクトにもアクセスするように指定できます。

In [24]:
df.describe(include=["object"])

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text
count,1052,1052,1052,1044,1044,1040,1052,1052,1044,1044,1052
unique,40,7,59,853,563,2,893,998,735,410,1043
top,,Physiology or Medicine,United States,7 November 1867,,male,http://en.wikipedia.org/wiki/Michael_Levitt,César Milstein,,,"Ernest Rutherford , born in New Zealand , Che..."
freq,910,250,350,4,362,982,4,2,29,409,2


ここから、受賞者には59の国籍があり、米国が350で最大のグループであるなどの情報が得られます。

また、生年月日は全部で1044ありますが、異なり数は853であることが分かります。複数の受賞者が生まれた縁起の良い日があるか、受賞者のデータが重複している可能性があります。

氏名の異なり数が998というのも、受賞者のデータが重複している可能性を示すものです。2つ以上の賞を受賞している人は数名いますが、54の重複はそれに対して多すぎます。

In [27]:
# もう一度、生データを確認

df.head(10)

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year
0,,Physiology or Medicine,Argentina,8 October 1927,24 March 2002,male,http://en.wikipedia.org/wiki/C%C3%A9sar_Milstein,César Milstein,"Bahía Blanca , Argentina","Cambridge , England","César Milstein , Physiology or Medicine, 1984",1984
1,Bosnia and Herzegovina,Literature,,9 October 1892,13 March 1975,male,http://en.wikipedia.org/wiki/Ivo_Andric,Ivo Andric *,"Dolac (village near Travnik), Austria-Hungary ...","Belgrade, SR Serbia, SFR Yugoslavia (present-d...","Ivo Andric *, born in then Austria–Hungary ,...",1961
2,Bosnia and Herzegovina,Chemistry,,"July 23, 1906",1998-01-07,male,http://en.wikipedia.org/wiki/Vladimir_Prelog,Vladimir Prelog *,"Sarajevo , Bosnia and Herzegovina , then part...","Zürich , Switzerland","Vladimir Prelog *, born in then Austria–Hung...",1975
3,,Peace,Belgium,None,None,None,http://en.wikipedia.org/wiki/Institut_de_Droit...,Institut de Droit International,None,None,"Institut de Droit International , Peace, 1904",1904
4,,Peace,Belgium,26 July 1829,6 October 1912,male,http://en.wikipedia.org/wiki/Auguste_Marie_Fra...,Auguste Beernaert,"Ostend , Netherlands (now Belgium )","Lucerne , Switzerland","Auguste Beernaert , Peace, 1909",1909
5,,Literature,Belgium,29 August 1862,6 May 1949,male,http://en.wikipedia.org/wiki/Maurice_Maeterlinck,Maurice Maeterlinck,"Ghent , Belgium","Nice , France","Maurice Maeterlinck , Literature, 1911",1911
6,,Peace,Belgium,22 April 1854,14 May 1943,male,http://en.wikipedia.org/wiki/Henri_La_Fontaine,Henri La Fontaine,Brussels,Belgium,"Henri La Fontaine , Peace, 1913",1913
7,,Physiology or Medicine,Belgium,13 June 1870,6 April 1961,male,http://en.wikipedia.org/wiki/Jules_Bordet,Jules Bordet,"Soignies, Belgium",,"Jules Bordet , Physiology or Medicine, 1919",1919
8,,Physiology or Medicine,Belgium,28 March 1892,18 July 1968,male,http://en.wikipedia.org/wiki/Corneille_Heymans,Corneille Heymans,"Ghent , Flanders","Knokke , Flanders","Corneille Heymans , Physiology or Medicine, 1938",1938
9,,Peace,Belgium,1910-02-10,1969-01-30,male,http://en.wikipedia.org/wiki/Georges_Pire,Georges Pire,"Dinant , Belgium","Leuven , Belgium","Georges Pire , Peace, 1958",1958


In [28]:
df.tail(5)

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year
1047,,Physics,Australia,"February 24, 1967",,male,http://en.wikipedia.org/wiki/Brian_P._Schmidt,Brian P. Schmidt,"Missoula, Montana",,"Brian P. Schmidt , born in the United States ...",2011
1048,,Peace,Argentina,"November 1, 1878","May 5, 1959",male,http://en.wikipedia.org/wiki/Carlos_Saavedra_L...,Carlos Saavedra Lamas,"Buenos Aires , Argentina","Buenos Aires, Argentina","Carlos Saavedra Lamas , Peace, 1936",1936
1049,,Physiology or Medicine,Argentina,1887-04-10,1971-09-21,male,http://en.wikipedia.org/wiki/Bernardo_Houssay,Bernardo Houssay,"Buenos Aires , Argentina","Buenos Aires , Argentina","Bernardo Houssay , Physiology or Medicine, 1947",1947
1050,,Chemistry,Argentina,1906-9-6,1987-12-2,male,http://en.wikipedia.org/wiki/Luis_Federico_Leloir,Luis Federico Leloir,"Paris , France","Buenos Aires , Argentina","Luis Federico Leloir , Chemistry, 1970",1970
1051,,Peace,Argentina,"November 26, 1931",,male,http://en.wikipedia.org/wiki/Adolfo_P%C3%A9rez...,Adolfo Pérez Esquivel,Buenos Aires,,"Adolfo Pérez Esquivel , Peace, 1980",1980


よく見ると、名前の後ろにアスタリスクがついている人がいたり、生年月日のフォーマットが統一されていなかったり、`born_in`の情報がほぼ存在しない、といったことが分かります。

ここまでの情報をもとに、現時点でのデータ仕様書を作っておきましょう。

前処理が終わった後にデータ仕様書をアップデートします。

In [17]:
# 現時点のデータ仕様書を作る

| カラム | カラムの説明 | データ型 | データ型（理想） | 尺度 | 値の説明 | NULL | UNIQ | CHECK |
|:--|:--|:--|:--|:--|:--|:--|:--|:--|
| born_in | 出生地 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| category | 賞の種類 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  | | 
| country | 国籍 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| date_of_birth | 生年月日 | テキスト | 数値 | 間隔尺度 |  | ✔ |  |  |
| date_of_death | 没年月日 | テキスト | 数値 | 間隔尺度 |  | ✔ |  |  |
| gender | 性別 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| link | ウィキペディアへのリンク | テキスト | テキスト | テキスト |  | ✔ |  |  |
| name | 氏名 | テキスト | テキスト | テキスト |  | ✔ |  |  |
| place_of_birth | 出生地 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| place_of_death | 死没地 | テキスト | カテゴリ | 名義尺度 |  | ✔ |  |  |
| text | 説明 | テキスト | テキスト | テキスト |  | ✔ |  |  |
| year | 受賞年 | 数値 | 数値 | 間隔尺度 |  | ✔ |  | 1901 $\leq$ x $\leq$ 2022 |

* `NULL`: 欠損値が存在する可能性があるか
* `UNIQ`: 重複しない値か
* `CHECK`: 満たすべき条件

In [20]:
# データの前処理（欠損値、重複、整列されていない行、不統一、エラー値、データの混合などに対処する）

In [34]:
# born_in

まずは、ほぼ存在しない`born_in`を適切に欠損値として処理できるようにします。

`apply`メソッドを使うと、セルに対して同じ処理を繰り返し実行できます。これを使って`born_in`の実際のデータ型（pandasのオブジェクトにラップされているもの）を見てみましょう。

In [32]:
df["born_in"].apply(type)

0       <class 'str'>
1       <class 'str'>
2       <class 'str'>
3       <class 'str'>
4       <class 'str'>
            ...      
1047    <class 'str'>
1048    <class 'str'>
1049    <class 'str'>
1050    <class 'str'>
1051    <class 'str'>
Name: born_in, Length: 1052, dtype: object

`set`を使って集合をとることで、異なり値だけを取り出します。

In [33]:
set(df["born_in"].apply(type))

{str}

すべてのデータがテキスト型であることが分かりました。何もないように見えるセルにも空白文字が入っているようです。

空白文字を欠損値（`NaN`）に置換しましょう。

In [38]:
df["born_in"].replace('', np.nan, inplace=True)

In [41]:
df["born_in"].count()

142

これで空白文字が欠損値に置換され、`born_in`の正しい数が分かるようになりました。

同じ要領で、データセット内のすべての空文字列を欠損値に置換します。

In [42]:
df.replace('', np.nan, inplace=True)

In [44]:
# name

さきほど、アスタリスクが付いた名前がありました。これを整えていきます。

まず、アスタリスクが付いた名前の数を調べます。

In [47]:
df[df.name.str.contains("\*")]["name"]

1                   Ivo Andric *
2              Vladimir Prelog *
14               Simon Kuznets *
15              Menachem Begin *
16                Shimon Peres *
                  ...           
1034            Martin Karplus *
1035    William Lawrence Bragg *
1039    Aleksandr M. Prokhorov *
1041     John Warcup Cornforth *
1046    Elizabeth H. Blackburn *
Name: name, Length: 142, dtype: object

142個ありました。アスタリスクを置換し、前後の空白を取ります。

In [52]:
df["name"] = df["name"].str.replace('*', '', regex=False)

In [53]:
df["name"] = df["name"].str.strip()

In [55]:
df[df.name.str.contains("\*")]["name"]

Series([], Name: name, dtype: object)

再度数えてみると、アスタリスク付きの名前がなくなっていることが分かります。

In [56]:
# 重複

次に、重複しているデータに対処します。`duplicated`メソッドで重複している行が`True`と判定されます。

In [62]:
df.duplicated("name", keep=False)

0        True
1       False
2        True
3       False
4       False
        ...  
1047     True
1048    False
1049    False
1050     True
1051    False
Length: 1052, dtype: bool

これをデータセット全体の条件にすると、氏名が重複している行のみを抽出できます。

In [64]:
all_dupes = df[df.duplicated("name", keep=False)]

In [65]:
all_dupes

,born_in,category,country,date_of_birth,date_of_death,gender,link,name,place_of_birth,place_of_death,text,year
0,NaN,Physiology or Medicine,Argentina,8 October 1927,24 March 2002,male,http://en.wikipedia.org/wiki/C%C3%A9sar_Milstein,César Milstein,"Bahía Blanca , Argentina","Cambridge , England","César Milstein , Physiology or Medicine, 1984",1984
2,Bosnia and Herzegovina,Chemistry,NaN,"July 23, 1906",1998-01-07,male,http://en.wikipedia.org/wiki/Vladimir_Prelog,Vladimir Prelog,"Sarajevo , Bosnia and Herzegovina , then part...","Zürich , Switzerland","Vladimir Prelog *, born in then Austria–Hung...",1975
11,NaN,Physiology or Medicine,Belgium,2 October 1917,4 May 2013,male,http://en.wikipedia.org/wiki/Christian_de_Duve,Christian de Duve,"Thames Ditton, Surrey, United Kingdom","Grez-Doiceau, Belgium","Christian de Duve , born in the United Kingdo...",1974
12,NaN,Chemistry,Belgium,25 January 1917,28 May 2003,male,http://en.wikipedia.org/wiki/Ilya_Prigogine,Ilya Prigogine,"Moscow , Russia","Brussels , Belgium","Ilya Prigogine , born in Russia , Chemistry, ...",1977
14,Belarus,Economics,NaN,"April 30, 1901","July 8, 1985",male,http://en.wikipedia.org/wiki/Simon_Kuznets,Simon Kuznets,Pinsk,"Cambridge, Massachusetts","Simon Kuznets *, born in then Russian Empire,...",1971
...,...,...,...,...,...,...,...,...,...,...,...,...
1039,Australia,Physics,NaN,11 July 1916,8 January 2002,male,http://en.wikipedia.org/wiki/Aleksandr_M._Prok...,Aleksandr M. Prokhorov,"Atherton, Queensland , Australia","Moscow , Russia","Aleksandr M. Prokhorov *, Physics, 1964",1964
1040,NaN,Literature,Australia,28 May 1912,30 September 1990,male,http://en.wikipedia.org/wiki/Patrick_White,Patrick White,"Knightsbridge , London","Sydney, Australia","Patrick White , born in the United Kingdom , ...",1973
1042,NaN,Economics,Australia,"May 29, 1920","August 9, 2000",male,http://en.wikipedia.org/wiki/John_Harsanyi,John Harsanyi,"Budapest , Hungary","Berkeley, California , USA","John Harsanyi , Economics, 1994",1994
1047,NaN,Physics,Australia,"February 24, 1967",NaN,male,http://en.wikipedia.org/wiki/Brian_P._Schmidt,Brian P. Schmidt,"Missoula, Montana",NaN,"Brian P. Schmidt , born in the United States ...",2011


299行の重複があることが分かりました。重複行を名前で並べ替え、国籍と受賞年を見てみます。

In [68]:
all_dupes.sort_values("name")[["name", "country", "year"]]

,name,country,year
131,Aaron Klug,United Kingdom,1982
121,Aaron Klug,South Africa,1982
106,Aaron Klug,NaN,1982
440,Alan MacDiarmid,United States,2000
742,Alan MacDiarmid,NaN,2000
...,...,...,...
417,William Vickrey,United States,1996
489,Yoichiro Nambu,United States,2008
773,Yoichiro Nambu,Japan,2008
1010,Zhores Ivanovich Alferov,NaN,2000


同じ年の受賞で異なる国籍が書かれている人がいます。今回参照したWikipediaでは、国ごとに受賞者のリストが作られているので、

## データの